# Chronist

*Long-term analysis of emotion, sentiment, and aging using photos and text.*

The goal of this project is to quantitatively monitor the emotional and physical changes of an individual over periods of time. My thesis is that if you can accurately show emotional or physical change over time, you can objectively pinpoint how an environmental change such as a career change, moving to a new city, starting or ending a relationship, or starting a new habit like going to the gym affected your physical and emotional health. This can lead to important insights on an individual level and for a population as a whole.

We start with two types of datasets. The first is a CSV of emotion, age, and ethnity as output by a photographic emotion detection library by Affectiva. The second type are CSVs of sentiment analysis data from iMessage, Facebook Messenger, 750 Words, and Day One. This data is all in the `/data` directory and further organized into directories based on each user - for example, `/user/a` refers to user `A`'s data. I've used letters to label users to protect their privacy. You can read more about the tools I've used to generate these CSVs [here](https://github.com/cjroth/chronist).

![](http://cjroth.com/images/content/lifeslice.png)

First, I'll import a few libraries. I'll use Pandas to clean up the data and then [Plotly](https://plot.ly/python/) to visualize it.

In [25]:
import os
import datetime
import numpy as np
import pandas as pd
import plotly
import plotly.offline as py

Next I tell Plotly and Python to use offline mode.

In [26]:
py.init_notebook_mode(connected=True)

I start by setting a few variables for later.

Our data is stored in the data directory:

In [27]:
data_directory = '../data'

I'm going to do this analysis on the first participant, `a`:

In [28]:
participant = 'a'

I'm going to use a 30-day moving average to show how emotion and sentiment changes over time:

In [29]:
rolling_mean_window = 30

I'm going to show data from April 2016 to February 2017, since that's the range of time that all of my datasets are available for:

In [30]:
timeframe = ('2016-04-01', '2017-08-14')

Next I'm going to define a function that removes outliers from the dataset. An outlier is defined as any value that is more than 1.5 times greater than the upper bound of the interquartile range or 1.5 times less than the lower bound of the interquartile range. I'll use this on each of the datasets that we import.

In [31]:
def remove_outliers(series):
    iqr = (series.quantile(0.25) * 1.5, series.quantile(0.75) * 1.5)
    outliers = (series < iqr[0]) | (series > iqr[1])
    return series[~outliers]

I'll also define a function to normalize the data such that all values are scaled to a range of -1 to 1.

In [32]:
def normalize(series):
    min = series.min()
    max = series.max()
    return ((series - min) / (max - min) - 0.5) * 2

Finally I'll create an empty dataframe to store the cleaned-up, normalized data.

In [33]:
data = pd.DataFrame()

Now that everything is set up for the analysis, I'll read the CSV into a dataframe and output the first few rows to see what kind of data I'm working with:

In [34]:
lifeslice = pd.read_csv(data_directory + '/' + participant + '/lifeslice.csv', parse_dates=[['date', 'time']], index_col=['date_time']).dropna()

lifeslice.head()

,path,faces,appearance.age,appearance.ethnicity,appearance.gender,appearance.glasses,emojis.disappointed,emojis.dominantEmoji,emojis.flushed,emojis.kissing,...,featurePoints.7.x,featurePoints.7.y,featurePoints.8.x,featurePoints.8.y,featurePoints.9.x,featurePoints.9.y,measurements.interocularDistance,measurements.orientation.pitch,measurements.orientation.roll,measurements.orientation.yaw
date_time,,,,,,,,,,,,,,,,,,,,,
2016-03-31 22:05:00,face_2016-03-31T22-05-00Z-0400.jpg,1,35 - 44,Caucasian,Male,No,0.001830,😗,0.001829,99.886192,...,474.336487,258.072327,513.724731,254.729645,548.703247,242.136078,132.422760,-11.550866,-0.858822,6.181901
2016-03-31 22:10:00,face_2016-03-31T22-10-00Z-0400.jpg,1,25 - 34,Caucasian,Male,No,0.001830,😐,0.001829,30.944508,...,434.028778,273.186523,479.393585,267.587677,509.938263,256.599731,131.829544,-4.173046,-16.175728,2.569681
2016-04-01 12:25:00,face_2016-04-01T12-25-00Z-0400.jpg,1,45 - 54,Caucasian,Male,No,0.001829,😐,0.001829,0.001833,...,656.819641,171.807587,732.909485,169.181442,777.705933,153.275681,206.592682,-16.416479,-4.150486,-9.395134
2016-04-01 12:35:00,face_2016-04-01T12-35-00Z-0400.jpg,1,45 - 54,Black African,Male,No,0.001829,😐,0.001829,0.001829,...,526.382080,191.286484,575.717590,196.858688,611.920288,195.521103,156.466446,1.059075,11.042427,-15.380928
2016-04-01 12:40:00,face_2016-04-01T12-40-00Z-0400.jpg,1,35 - 44,Hispanic,Male,No,0.001831,😐,0.001829,0.002322,...,495.512024,414.139648,526.654053,410.078979,553.704895,401.493317,98.379379,-15.917084,-6.357203,6.252743


Now for the fun part. I'm going to remove outliers from each of the datasets, normalize them all to a scale of -1 to 1, and then put the columns of interest into a single dataframe.

In [35]:
series = lifeslice['emotions.valence']
series = remove_outliers(series)
series = normalize(series)
data = data.merge(series.to_frame('lifeslice'), how='outer', left_index=True, right_index=True)

for dataset in ['imessage', 'facebook', 'dayone', '750words']:
    csv = data_directory + '/' + participant + '/' + dataset + '.csv'
    if (not os.path.exists(csv)):
        continue
    df = pd.read_csv(csv, parse_dates=[['date', 'time']], index_col=['date_time']).dropna()
    series = df['sentiment.comparative']
    series = remove_outliers(series)
    series = normalize(series)
    data = data.merge(series.to_frame(dataset), how='outer', left_index=True, right_index=True)

And I'll show the first few rows to see what that result looks like:

In [36]:
data.head()

,lifeslice,imessage,facebook,dayone
date_time,,,,
2007-05-13 19:22:32,NaN,NaN,NaN,-0.480245
2007-12-30 06:51:05,NaN,NaN,NaN,-0.337815
2007-12-31 18:10:50,NaN,NaN,NaN,0.576631
2008-01-27 07:09:58,NaN,NaN,NaN,0.340136
2008-02-09 06:07:29,NaN,NaN,NaN,-0.204040


Most of the rows have values for only one column, but this makes sense. Unless the participant had sent an iMessage at exactly the same moment that Lifeslice took a photo of them, there would only be a single value for that time. I'm going to resample the data later to re-distribute the data into evenly-spaced time buckets so that each time has a value.

For now, I'll just take a slice of the dataframe with only the timeframe that I want to analyze.

In [37]:
start, end = (data.index.searchsorted(datetime.datetime.strptime(i, '%Y-%m-%d')) for i in timeframe)
data = data[start:end]

And then I'll show a histogram of the data to see how it's distributed:

In [38]:
fig = plotly.tools.make_subplots(rows=len(data.columns), cols=1)

for index, column in enumerate(data.columns):
    trace = plotly.graph_objs.Histogram(
        name = column,
        x = data[column],
    )
    fig.append_trace(trace, index + 1, 1)

fig['layout'].update(height=len(data.columns) * 250)
plot_url = py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



It looks like there a dramatically unproportionate amount of normalized sentiment scores that are -1 and Lifeslice valence scores that are 1 which is skewing the data. I'll remove those rows and try again:

In [39]:
for column in data.columns:
    if column == 'lifeslice':
        data = data[data[column] != 1]
        continue
    data = data[data[column] != -1]

Now I'll create a set of histograms again and see if it looks any better:

In [40]:
fig = plotly.tools.make_subplots(rows=len(data.columns), cols=1)

for index, column in enumerate(data.columns):
    trace = plotly.graph_objs.Histogram(
        name = column,
        x = data[column],
    )
    fig.append_trace(trace, index + 1, 1)

fig['layout'].update(height=len(data.columns) * 250)
plot_url = py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



Sweet, that looks way better. The distribution looks much more reasonable now.

Now it's time to resample. Basically I'm going to squeeze all of these random events that occur at a specific moment in time into 1-day time buckets by averaging all of the seconds for each day together to create an average for that day. This is called resampling. Luckily Pandas makes this very easy with the `resample` method. Then I'll output the first few rows to see how the result looks:

In [41]:
rule = '1d'
resampled = data.resample('1d').mean().fillna(data.mean()).rolling(rolling_mean_window, center=True).mean()
resampled.head()

start, end = tuple(datetime.datetime.strptime(i, '%Y-%m-%d') for i in timeframe)
data = data[data.index.searchsorted(start):data.index.searchsorted(end)]

The first and last 15 days of the timeframe are all `NaN` because of the way resampling works. It will only calculate a mean for a day if it has a full 30 days of values before and after the day, which means it leaves the first and least 30 days out. I'll drop these rows to make things cleaner.

In [42]:
resampled.dropna(inplace=True)
resampled.head()

,lifeslice,imessage,facebook,dayone
date_time,,,,
2016-04-16,0.666027,0.180028,-0.029499,0.069100
2016-04-17,0.665517,0.181921,-0.027027,0.057825
2016-04-18,0.671120,0.186216,-0.048440,0.057825
2016-04-19,0.675272,0.172122,-0.039695,0.057825
2016-04-20,0.678647,0.182128,-0.039695,0.034981


I'm almost ready to chart the data, but I want to chart not only the moving averages, but also the upper and lower quartiles so that I can get a sense of how much variation there is in the data.

In [43]:
lower = data.resample(rule).apply(lambda x: x.quantile(q=0.25)).fillna(data.mean()).rolling(rolling_mean_window, center=True).mean().dropna()
upper = data.resample(rule).apply(lambda x: x.quantile(q=0.75)).fillna(data.mean()).rolling(rolling_mean_window, center=True).mean().dropna()

Cool. Now it's finally time to plot the data. I'll use Plotly to create four "lines", which Plotly more accurately calls `traces`. The first trace will be to show all of the actual data points. This will help me see how the moving averages relate to the actual data. The second trace will be the moving average itself, and the third and fourth trace will together create the lower and upper bounds.

In [44]:
colors = [
    '#50514F',
    '#F25F5C',
    '#FFE066',
    '#247BA0',
    '#70C1B3',
]

datasets = [[
        
    # Scatterplot
    plotly.graph_objs.Scatter(
        name = column,
        x = data.index,
        y = data[column],
        mode = 'markers',
        marker = {
            'size': 1,
            'color': colors[index],
        },
    ),
        
    # Moving average
    plotly.graph_objs.Scatter(
        name = column + ' ma',
        x = resampled.index,
        y = resampled[column],
        mode = 'lines',
        fill = 'tonexty',
        fillcolor = 'rgba(68, 68, 68, 0.3)',
        line = {
            'color': colors[index],
        },
    ),
        
    # Lower quartile
    plotly.graph_objs.Scatter(
        name = column,
        x = resampled.index,
        y = lower[column],
        line = dict(width = 0),
        showlegend = False,
        mode = 'lines',
    ),
        
    # Upper quartile
    plotly.graph_objs.Scatter(
        name = column,
        x = resampled.index,
        y = upper[column],
        fill='tonexty',
        fillcolor='rgba(68, 68, 68, 0.3)',
        marker=dict(color = '444'),
        line=dict(width = 0),
        showlegend = False,
        mode='lines',
    )
        
] for index, column in enumerate(data.columns)]

I'll start by showing the Lifeslice emotional analysis scores and the iMessage sentiment scores on the same plot so that I can visually compare them.

In [45]:
to_plot = [trace for dataset in datasets[0:2] for trace in dataset]
py.iplot(to_plot)

And now I'll show each of the datasets independently:

In [46]:
fig = plotly.tools.make_subplots(rows=len(datasets), cols=1)

for index, dataset in enumerate(datasets):
    for trace in dataset:
        fig.append_trace(trace, index + 1, 1)

fig['layout'].update(title='Sentiment Comparisons', height=len(datasets) * 250)
plot_url = py.iplot(fig, filename='stacked-subplots')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



That looks kind of neat. It makes it easy to visually get a sense of how participant `a`'s emotion and sentiment scores changed over time, but it's not very clear if there are any correlations. I'll check directly:

In [47]:
resampled.corr()

,lifeslice,imessage,facebook,dayone
lifeslice,1.000000,-0.461035,0.193094,-0.140937
imessage,-0.461035,1.000000,-0.131269,0.190128
facebook,0.193094,-0.131269,1.000000,0.059101
dayone,-0.140937,0.190128,0.059101,1.000000


It looks like there is a slight negative correlation between iMessage and Lifeslice. I'll plot this to investigate:

In [48]:
trace = plotly.graph_objs.Scatter(
    name = 'comparison',
    x = resampled['lifeslice'],
    y = resampled['imessage'],
    mode = 'markers',
    marker = {
        'size': 5,
        'color': colors[0],
    },
)
py.iplot([trace])

Indeed, it is slight.